<a href="https://colab.research.google.com/github/victorialovefranklin/Toward-Climate-Resilient-Energy-Systems/blob/main/EAGLE_I_DATA__CLEANING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **EAGLE-I (2014-2023) Data Integration & Cleaning**

## **Task:** Load the 10 files Environment for Analysis of Geo-Located Energy Information (EAGLE-I) datasets across 2014 to 2023

In [77]:
# ==============================
# EAGLE-I (2014–2023) — Data Integration & Cleaning
# ==============================

import os
import warnings
import pandas as pd
import numpy as np

warnings.filterwarnings("ignore")

# -------- Explicit file list (each year, 2014–2023) --------
YEAR_TO_PATH = {
    2014: "/content/eaglei_outages_2014.csv",
    2015: "/content/eaglei_outages_2015.csv",
    2016: "/content/eaglei_outages_2016.csv",
    2017: "/content/eaglei_outages_2017.csv",
    2018: "/content/eaglei_outages_2018.csv",
    2019: "/content/eaglei_outages_2019.csv",
    2020: "/content/eaglei_outages_2020.csv",
    2021: "/content/eaglei_outages_2021.csv",
    2022: "/content/eaglei_outages_2022.csv",
    2023: "/content/eaglei_outages_2023.csv",
}

CLEANED_OUTPUT_FILE = "/content/eaglei_outages_cleaned.csv"
# FIPS_FILE = "/content/CA_County_FIPS.csv" # Remove FIPS file path

print("Planned files for analysis (2014–2023):")
for y, p in YEAR_TO_PATH.items():
    print(f" - {y}: {p}")

# -------- Load and Concatenate Data --------
frames = []
file_report = []   # to summarize loads per file
missing_files = []

for yr, fp in YEAR_TO_PATH.items():
    if os.path.exists(fp):
        try:
            df_i = pd.read_csv(fp, low_memory=False)
            # force year column to exist and be int
            if "year" not in df_i.columns:
                df_i["year"] = yr
            else:
                df_i["year"] = pd.to_numeric(df_i["year"], errors="coerce").fillna(yr).astype(int)

            df_i["__source_file"] = os.path.basename(fp)
            frames.append(df_i)
            file_report.append({"year": yr, "path": fp, "loaded_rows": len(df_i), "status": "OK"})
            if yr == 2023: # Add this print statement for clarity
                print(f"[INFO] Successfully processed 2023 file: {fp}")
        except Exception as e:
            file_report.append({"year": yr, "path": fp, "loaded_rows": 0, "status": f"ERROR: {e}"})
    else:
        missing_files.append(fp)
        file_report.append({"year": yr, "path": fp, "loaded_rows": 0, "status": "MISSING"})

# Print a quick report
print("\n=== File Load Report ===")
if file_report:
    rep = pd.DataFrame(file_report).sort_values("year")
    print(rep.to_string(index=False))
else:
    print("No files were checked.")

if missing_files:
    print("\n[WARN] Missing files (not found on disk):")
    for m in missing_files:
        print(f" - {m}")

if not frames:
    raise RuntimeError("No CSVs loaded for years 2014–2023. Check file paths or file availability.")

# Concatenate and enforce year range
df = pd.concat(frames, ignore_index=True)
df = df[(df["year"] >= 2014) & (df["year"] <= 2023)].copy()
df.drop_duplicates(inplace=True)
print(f"\n[OK] Loaded {len(df):,} rows across {len(frames)} files (2014–2023).")

# -------- Dynamic Column Detection --------
customers_candidates = ["customers_out", "customers", "cust_out", "customers_outage", "outage_customers"]
fips_candidates      = ["county_fips", "fips", "fips_code", "county_fips_code", "GEOID"]
county_name_candidates = ["county", "county_name"]
lat_candidates       = ["lat", "latitude", "y", "Lat", "Latitude"]
lon_candidates       = ["lon", "longitude", "x", "Lon", "Longitude", "long"]
time_candidates      = ["run_start_time", "start_time", "time", "timestamp", "datetime"]

def first_existing(frame, candidates):
    return next((c for c in candidates if c in frame.columns), None)

CUSTOMERS_COL   = first_existing(df, customers_candidates)
FIPS_COL        = first_existing(df, fips_candidates)
COUNTY_NAME_COL = first_existing(df, county_name_candidates)
LAT_COL         = first_existing(df, lat_candidates)
LON_COL         = first_existing(df, lon_candidates)
TIME_COL        = first_existing(df, time_candidates)

# Key sanity check: customers + year must exist
required_any = [CUSTOMERS_COL, "year"]
if any(col is None for col in required_any):
    raise ValueError(f"Required columns missing. Detected columns: customers={CUSTOMERS_COL}, year={'year' in df.columns}")

# -------- Data Cleaning --------
key_cols = [c for c in [FIPS_COL, CUSTOMERS_COL, "year", TIME_COL] if c is not None]
before = len(df)
df_cleaned = df.dropna(subset=key_cols).copy()
print(f"[OK] Dropped rows with missing values in key columns ({before - len(df_cleaned):,} rows removed).")

# Standardize FIPS if present
if FIPS_COL in df_cleaned.columns:
    df_cleaned[FIPS_COL] = (
        df_cleaned[FIPS_COL]
        .astype(str)
        .str.replace(r"\.0$", "", regex=True)
        .str.zfill(5)
    )
    print(f"[OK] Cleaned and formatted '{FIPS_COL}' as 5-digit strings.")

# Standardize time if present
if TIME_COL in df_cleaned.columns:
    df_cleaned[TIME_COL] = pd.to_datetime(df_cleaned[TIME_COL], errors="coerce", utc=False)
    before_time = len(df_cleaned)
    df_cleaned.dropna(subset=[TIME_COL], inplace=True)
    print(f"[OK] Converted '{TIME_COL}' to datetime. Removed {before_time - len(df_cleaned):,} rows with invalid timestamps.")

# Ensure customers numeric
df_cleaned[CUSTOMERS_COL] = pd.to_numeric(df_cleaned[CUSTOMERS_COL], errors="coerce").fillna(0)

# Final output columns for cleaned data
output_cols = [col for col in [FIPS_COL, COUNTY_NAME_COL, CUSTOMERS_COL, "year", TIME_COL, "__source_file"] if col and col in df_cleaned.columns]
df_cleaned = df_cleaned[output_cols].copy()


# -------- Per-year counts to confirm inclusion --------
# This table shows the number of rows loaded per year after cleaning.
print("\n=== Rows Loaded Per Year (post-cleaning) ===") # Label for the post-cleaning table
if CUSTOMERS_COL and CUSTOMERS_COL in df_cleaned.columns:
    year_counts = df_cleaned.groupby("year")[CUSTOMERS_COL].size().rename("rows_loaded").reset_index()
    print(year_counts.to_string(index=False))
else:
    print("Cannot generate per-year counts: Customers column not found.")


# -------- Export Cleaned Data --------
# Ensure the directory exists before exporting
export_dir = os.path.dirname(CLEANED_OUTPUT_FILE)
if export_dir and not os.path.exists(export_dir):
    os.makedirs(export_dir)
    print(f"[INFO] Created directory: {export_dir}")

try:
    df_cleaned.to_csv(CLEANED_OUTPUT_FILE, index=False)
    print(f"\n[OK] Cleaned data exported to: {CLEANED_OUTPUT_FILE}")
except Exception as e:
    print(f"[WARN] Could not export cleaned data to CSV: {e}")

print("\n✅ Complete: Data integration and cleaning for EAGLE-I outage data (2014–2023).")

Planned files for analysis (2014–2023):
 - 2014: /content/eaglei_outages_2014.csv
 - 2015: /content/eaglei_outages_2015.csv
 - 2016: /content/eaglei_outages_2016.csv
 - 2017: /content/eaglei_outages_2017.csv
 - 2018: /content/eaglei_outages_2018.csv
 - 2019: /content/eaglei_outages_2019.csv
 - 2020: /content/eaglei_outages_2020.csv
 - 2021: /content/eaglei_outages_2021.csv
 - 2022: /content/eaglei_outages_2022.csv
 - 2023: /content/eaglei_outages_2023.csv
[INFO] Successfully processed 2023 file: /content/eaglei_outages_2023.csv

=== File Load Report ===
 year                             path  loaded_rows status
 2014 /content/eaglei_outages_2014.csv        25856     OK
 2015 /content/eaglei_outages_2015.csv        30748     OK
 2016 /content/eaglei_outages_2016.csv        24333     OK
 2017 /content/eaglei_outages_2017.csv        24578     OK
 2018 /content/eaglei_outages_2018.csv        17897     OK
 2019 /content/eaglei_outages_2019.csv        57796     OK
 2020 /content/eaglei_outag

# **Separate State and County FIPS Codes (California = 6)**

# T**ask: Load the /content/eaglei_outages_cleaned.csv**

In [78]:
import pandas as pd
import csv # Import the csv module

# ---- 1. Load the cleaned EAGLE-I dataset ----
file_path = "/content/eaglei_outages_cleaned.csv"  # adjust if using a different path
df = pd.read_csv(file_path)

# ---- 2. Detect the FIPS column automatically ----
fips_columns = [col for col in df.columns if "fips" in col.lower()]
print("Detected FIPS-related columns:", fips_columns)

# Use the first detected FIPS column (e.g., 'fips_code')
target_col = fips_columns[0] if fips_columns else None
if not target_col:
    raise ValueError("No FIPS column detected. Please check your dataset.")

# ---- 3. Clean and standardize FIPS values ----
df[target_col] = (
    df[target_col]
    .astype(str)
    .str.strip()
    .str.replace(r"\.0$", "", regex=True)  # remove any ".0"
    .str.replace(r"[^0-9]", "", regex=True)  # keep digits only
)

# ---- 4. Split into State and County FIPS columns ----
df["state_fips"] = df[target_col].str[0].str.zfill(2)   # first digit (CA = 06)
df["county_fips"] = df[target_col].str[1:].str.zfill(3) # remaining 3 digits

# Convert county_fips to string explicitly before export to help preserve leading zeros
df["county_fips"] = df["county_fips"].astype(str)


# ---- 5. Export to a new CSV ----
output_path = "/content/eaglei_outages_cleaned_with_state_county_fips.csv"
# Use quoting=csv.QUOTE_NONNUMERIC to help preserve leading zeros in non-numeric columns
df.to_csv(output_path, index=False, quoting=csv.QUOTE_NONNUMERIC)

print(f"\n✅ Successfully created 'state_fips' and 'county_fips' columns.")
print(f"Saved to: {output_path}")

# ---- 6. Generate Report of Results (Analysis by County FIPS) ----
print("\n=== Analysis Report: Outages by County FIPS ===")

if "county_fips" in df.columns and "customers_out" in df.columns:
    # Group by county_fips and sum customers_out and count outages
    outages_by_county_fips = df.groupby("county_fips")["customers_out"].agg(['sum', 'count']).reset_index()
    outages_by_county_fips.rename(columns={'sum': 'total_customers_out', 'count': 'number_of_outages'}, inplace=True)

    # Sort by total customers affected for summary
    outages_by_county_fips_sorted = outages_by_county_fips.sort_values("total_customers_out", ascending=False)

    print("\nTop 15 Counties by Total Customers Affected (by County FIPS):")
    display(outages_by_county_fips_sorted.head(15))

    print("\nTop 15 Counties by Number of Outages (by County FIPS):")
    display(outages_by_county_fips.sort_values("number_of_outages", ascending=False).head(15))

else:
    print("\nCannot generate report: 'county_fips' or 'customers_out' column not found after processing.")

print("\n✅ Analysis Report Generated.")

Detected FIPS-related columns: ['fips_code']

✅ Successfully created 'state_fips' and 'county_fips' columns.
Saved to: /content/eaglei_outages_cleaned_with_state_county_fips.csv

=== Analysis Report: Outages by County FIPS ===

Top 15 Counties by Total Customers Affected (by County FIPS):


,county_fips,total_customers_out,number_of_outages
33,067,38997105.0,14335
18,037,36861943.0,22012
43,087,11762455.0,6519
35,071,11546927.0,18327
55,111,11368373.0,15309
36,073,10076272.0,19198
38,077,9928191.0,6387
42,085,9548932.0,7843
29,059,9251835.0,17762
28,057,8699807.0,4712



Top 15 Counties by Number of Outages (by County FIPS):


,county_fips,total_customers_out,number_of_outages
18,037,36861943.0,22012
36,073,10076272.0,19198
35,071,11546927.0,18327
32,065,7352065.0,17847
29,059,9251835.0,17762
55,111,11368373.0,15309
33,067,38997105.0,14335
56,113,6349503.0,13266
14,029,2254911.0,12435
53,107,1926913.0,12135



✅ Analysis Report Generated.
